<a href="https://colab.research.google.com/github/Onitsiky/MBD-Assignment-1/blob/main/MBD_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 1

##Assignment Overview

Total Points: 100

Duration: 2 weeks (Tuesday Jan 20th, 6 am  - Wednesday Feb 04rd,6am)

Deliverables:

  - Jupyter notebook (.ipynb) or Module(.py) with answers and visualizations
  - README explaining your approach(2 pages)
    - Overview of your approach
    - Key decisions you made (e.g., how you handled missing values)
    - Summary of key findings
  - All submissions should be hosted on your github account.(submit the link to the repo alongside your homework)


###TASK 1: Load and Merge data

1. Load 3 SMS/call/internet activity data (sms-call-internet-mi-2013-11-02.csv,
sms-call-internet-mi-2013-11-04.csv, sms-call-internet-mi-2013-11-06.csv) and combine them.

2. Create a clean, analysis-ready dataset.

####Requirements:
- Load all 3 activity files
- Add date and time columns
- Combine into single dataframe
- Handle any missing values (Hint: Fill with mean)
- Add aggregate columns (total_sms, total_calls, total_internet)

####Questions (50 points):
- How many total records across all 3 datasets? (10)
- How many unique grid squares (CellID) are there? (5)
- How many unique country codes appear in the data? (5)
- Are there any missing values?
  - If yes, input missing/NaN with the mean of each columns. (10)
  - For which columns are missing values most common? (5)
  - How many records did you modify? (5)

- What is the most common peak hour across all grids? (5)
  - What hour has the lowest activity? (5)
  - Calculate and report: mean, median, std, min, max for total calls by hour (15)

- What percentage of total activity happens during daytime (6am-8pm) vs nighttime(8 pm -6 am)? (5)


- Do international calls happen at different times than domestic? (compare hourly patterns) (5)

- Use numpy to perform statistical comparisons between different conditions. (5)

  Compare:
    1. Domestic (Italy) vs International calls.
        - What percentage of calls are international vs domestic? (5)
        - What percentage of SMS are international vs domestic? (5)
        - Are international calls more incoming or outgoing (calculate the ratio) (5)
    2. Different activity types.
        - Is there a correlation between SMS volume and Call volume at the grid level? (5)

In [1]:
from google.colab import userdata
import os

# Store your credentials in Colab Secrets first (see below)
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

!kaggle datasets download -d marcodena/mobile-phone-activity #marcodena/mobile-phone-activity
!unzip mobile-phone-activity.zip


Dataset URL: https://www.kaggle.com/datasets/marcodena/mobile-phone-activity
License(s): DbCL-1.0
 55% 133M/242M [00:00<00:00, 1.39GB/s]
100% 242M/242M [00:00<00:00, 711MB/s] 
Archive:  mobile-phone-activity.zip
  inflating: ISTAT_census_variables_2011.csv  
  inflating: Italian_provinces.geojson  
  inflating: mi-to-provinces-2013-11-01.csv  
  inflating: mi-to-provinces-2013-11-02.csv  
  inflating: mi-to-provinces-2013-11-03.csv  
  inflating: mi-to-provinces-2013-11-04.csv  
  inflating: mi-to-provinces-2013-11-05.csv  
  inflating: mi-to-provinces-2013-11-06.csv  
  inflating: mi-to-provinces-2013-11-07.csv  
  inflating: milano-grid.geojson     
  inflating: sms-call-internet-mi-2013-11-01.csv  
  inflating: sms-call-internet-mi-2013-11-02.csv  
  inflating: sms-call-internet-mi-2013-11-03.csv  
  inflating: sms-call-internet-mi-2013-11-04.csv  
  inflating: sms-call-internet-mi-2013-11-05.csv  
  inflating: sms-call-internet-mi-2013-11-06.csv  
  inflating: sms-call-internet-mi-

In [2]:
import pandas as pd
import numpy as np

# TASK 1: Load and Merge data
## 1. Loading 3 SMS/call/internet activity data

In [4]:
cdr1_df = pd.read_csv('sms-call-internet-mi-2013-11-02.csv')
cdr2_df = pd.read_csv('sms-call-internet-mi-2013-11-04.csv')
cdr3_df = pd.read_csv('sms-call-internet-mi-2013-11-06.csv')

cdr_df_raw = pd.concat([cdr1_df, cdr2_df, cdr3_df], ignore_index=True)

cdr_df_raw.head()

,datetime,CellID,countrycode,smsin,smsout,callin,callout,internet
0,2013-11-02 00:00:00,1,0,0.2445,NaN,NaN,NaN,NaN
1,2013-11-02 00:00:00,1,39,1.4952,1.1213,0.2708,0.3004,46.5094
2,2013-11-02 00:00:00,1,53,0.0018,0.0036,NaN,NaN,NaN
3,2013-11-02 00:00:00,2,0,0.2458,NaN,NaN,NaN,NaN
4,2013-11-02 00:00:00,2,39,1.5028,1.1243,0.2751,0.3023,46.6933


## 2. Creating a clean, analysis-ready dataset.

In [29]:
cdr_df = cdr_df_raw.copy()

# Adding date and time columns
cdr_df['datetime'] = pd.to_datetime(cdr_df['datetime'])

cdr_df['date'] = cdr_df['datetime'].dt.date
cdr_df['time'] = cdr_df['datetime'].dt.time
cdr_df['hour'] = cdr_df['datetime'].dt.hour

# Handling missing values
cols_to_handle = ['smsin', 'smsout', 'callin', 'callout', 'internet']

cdr_df[cols_to_handle] = cdr_df[cols_to_handle].fillna(cdr_df[cols_to_handle].mean())

# Adding aggregate columns
cdr_df['total_sms'] = cdr_df['smsin'] + cdr_df['smsout']
cdr_df['total_calls'] = cdr_df['callin'] + cdr_df['callout']
cdr_df['total_internet'] = cdr_df['internet']

cdr_df.head()

,datetime,CellID,countrycode,smsin,smsout,callin,callout,internet,date,time,hour,total_sms,total_calls,total_internet
0,2013-11-02,1,0,0.2445,7.440751,7.738604,5.686694,101.976153,2013-11-02,00:00:00,0,7.685251,13.425298,101.976153
1,2013-11-02,1,39,1.4952,1.121300,0.270800,0.300400,46.509400,2013-11-02,00:00:00,0,2.616500,0.571200,46.509400
2,2013-11-02,1,53,0.0018,0.003600,7.738604,5.686694,101.976153,2013-11-02,00:00:00,0,0.005400,13.425298,101.976153
3,2013-11-02,2,0,0.2458,7.440751,7.738604,5.686694,101.976153,2013-11-02,00:00:00,0,7.686551,13.425298,101.976153
4,2013-11-02,2,39,1.5028,1.124300,0.275100,0.302300,46.693300,2013-11-02,00:00:00,0,2.627100,0.577400,46.693300


# Questions (50 points):
* How many total records across all 3 datasets?

In [10]:
print(f'Across all 3 datasets, there are {cdr_df.shape[0]} records.')

Across all 3 datasets, there are 6564031 records.


* How many unique grid squares (CellID) are there?

In [11]:
print(f'There are {cdr_df['CellID'].nunique()} unique grid squares.')

There are 10000 unique grid squares.


* How many unique country codes appear in the data?

In [12]:
print(f'There are {cdr_df['countrycode'].nunique()} unique country codes in the data.')

There are 302 unique country codes in the data.


* Are there any missing values?
  * Yes there are missing values in several columns that have been filled which the columns mean
  * For which columns are missing values most common ?

In [26]:
print(cdr_df_raw.isnull().sum().sort_values(ascending=False))

smsout         5025738
callin         4761685
smsin          3964171
callout        3764484
internet       3621117
countrycode          0
datetime             0
CellID               0
dtype: int64


From the output above, we can see that the most missing values are from the columns **smsout** and **callin**.
* How many records did you modify ?

In [22]:
print(f'{cdr_df_raw.isnull().any(axis=1).sum()} records have been modified.')

5880441 records have been modified.


* What is the most common peak hour across all grids? (5)

  * What hour has the lowest activity

In [33]:
cdr_df['total_activity'] = cdr_df['total_calls'] + cdr_df['total_sms'] + cdr_df['total_internet']

activity_summary_per_hour = cdr_df.groupby('hour')['total_activity'].sum()
print(f'Most common peak hour: {activity_summary_per_hour.idxmax()}:00')
print(f'Hour with the lowest activity: {activity_summary_per_hour.idxmin()}:00')

Most common peak hour: 17:00
Hour with the lowest activity: 4:00


  * Calculate and report: mean, median, std, min, max for total calls by hour

In [50]:
hourly_calls = cdr_df.groupby('hour')['total_calls'].sum()

hourly_calls_stats = hourly_calls.describe()
print('===== Statistics for total calls by hour =====')
print(f'      mean:      {hourly_calls_stats['mean']}')
print(f'      median:    {hourly_calls_stats['50%']}')
print(f'      std:       {hourly_calls_stats['std']}')
print(f'      min:       {hourly_calls_stats['min']}')
print(f'      max:       {hourly_calls_stats['max']}')

===== Statistics for total calls by hour =====
      mean:      3671836.305684332
      median:    4145010.3328908295
      std:       1876942.9077285146
      min:       971110.8313857738
      max:       5801570.122925039


* What percentage of total activity happens during daytime (6am-8pm) vs nighttime(8 pm -6 am)?

In [56]:
cdr_df['is_daytime'] = (cdr_df['hour'] >= 6) & (cdr_df['hour'] < 20)

daytime_activity = cdr_df[cdr_df['is_daytime']]['total_activity'].sum()
nighttime_activity = cdr_df[~cdr_df['is_daytime']]['total_activity'].sum()
total = daytime_activity + nighttime_activity

print(f'Daytime activity: {round(daytime_activity * 100/total, 2)}%')
print(f'Daytime activity: {round(nighttime_activity* 100/total, 2)}%')

Daytime activity: 73.55%
Daytime activity: 26.45%


* Do international calls happen at different times than domestic? (compare hourly patterns)

In [60]:
cdr_df['is_domestic'] = cdr_df['countrycode'] ==39

pattern = cdr_df.groupby(['hour', 'is_domestic'])['total_calls'].sum().unstack()

pattern_norm = pattern / pattern.max()

print("Normalized Hourly Call Patterns (Domestic vs International):")
print(pattern_norm.head())

Normalized Hourly Call Patterns (Domestic vs International):
is_domestic     False     True 
hour                           
0            0.411799  0.056617
1            0.319102  0.039455
2            0.267480  0.046940
3            0.246476  0.052836
4            0.284774  0.052263
